# IBM Robustness Toolbox

* More Infos: https://github.com/sgxcj777/Adversarial-testing-toolbox-with-CLEVER

* PLAN: Morgen das hier noch schöner machen (ordentlich alle funktionen, sodass alles läuft, committen).
    * checken, dass playground noch geht
    * hyperparam search
* und dann bis Sonntag: mehrere Attacks & defenses probieren.
* -> dann haben wir mehrere Attacks, mehrere Training-Mechanismen. Ist dann doch top!!!

* dann: v.a. Training & Evaluation anwenden.





### More Frameworks
* Foolbox: 1.5K -> aber eher nur attacks...
* TF-Cleverhans: 4.6K
* IBM ART: 1.6K

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
sys.path.append("/nfs/homedirs/herrmanp/clevertest/adversarial-robustness-toolbox")
import art

In [4]:
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_lightning import loggers
import config


from classification.models.M5 import M5, M5PLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule

from utils.RobustnessExperiment import load_module
from utils.Visual import show_audio, show_spectogram, show_waveplot

/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [5]:
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

In [24]:
#model = load_module("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/SpectrogramCNN/optuna_90.5.p", SpectrogramCNNPLModule)
#model = load_module("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_vanilla.p", M5PLModule)
model = load_module("/nfs/homedirs/herrmanp/shortcut/SAVED_MODELS/M5/vanilla_model.p", M5PLModule)
model.prepare_data()
datasetHandler.load(model, 'training')
datasetHandler.load(model, 'validation')
model.cuda()
model.report()

Accuracy: 	0.84
Precision: 	0.89
Recall: 	0.77
F1-Score: 	0.83

VAL-ACC: 1424/1687 (84.41019561351511%)

P-Rate: 	0.42


{'tp': array(638),
 'fp': array(75),
 'tn': array(786),
 'fn': array(188),
 'correct': array(1424),
 'n': 1687,
 'acc': 0.8441019561351512,
 'prec': 0.8948106591865358,
 'rec': 0.7723970944309927,
 'f1': 0.8291098115659519,
 'attack_args': None,
 'p_rate': 0.4226437462951986}

In [8]:
x_test = model.val_dataloader().dataset[0:100]
x_test = []
y_test = []
for i in range(100):
    x_test.append(model.val_dataloader().dataset[i]["audio"])
    y_test.append(model.val_dataloader().dataset[i]["label"])



In [9]:
import torch.nn.functional as F

def cross_entropy_one_hot(inp, target):
    _, labels = target.max(dim=1)
    return F.cross_entropy(inp, labels)

opt = torch.optim.Adam(model.model.parameters(), lr = 1e-4)

from art.estimators.classification.pytorch import PyTorchClassifier
modelX = PyTorchClassifier(model =  model.model,
        loss = cross_entropy_one_hot,
        input_shape = (1, 8000),
        nb_classes = 2,
        channels_first = True,
        optimizer = opt)

In [28]:
scores = []
for i in range(10):
    x = model.val_dataloader().dataset[i]["audio"].numpy()
    out = art.metrics.clever_u(classifier = modelX, x = x, nb_batches = 50, batch_size = 5, radius = 1, norm = np.inf, c_init = 1.0, pool_factor = 100)
    print(out)
    scores.append(out)
print("AVG OVER 10 SAMPLES: {}".format(sum(scores)/10))

CLEVER untargeted:   0%|          | 0/1 [00:00<?, ?it/s]

0.008684896680564111


CLEVER untargeted:   0%|          | 0/1 [00:00<?, ?it/s]

0.022867354281745536


CLEVER untargeted:   0%|          | 0/1 [00:00<?, ?it/s]

...

0.018434056837202693


CLEVER untargeted:   0%|          | 0/1 [00:00<?, ?it/s]

0.022491356462704885


CLEVER untargeted: 100%|██████████| 1/1 [00:01<00:00,  1.55s/it]

0.008857220808815404
AVG OVER 10 SAMPLES: 0.01857831601944337


In [11]:
x_test_arr = [t.numpy() for t in x_test]
y_test_arr = [t for t in y_test]


total = np.vstack(x_test_arr)
totaly = np.array(y_test_arr)
print(total.shape)

b = np.zeros((totaly.size, totaly.max()+1))
b[np.arange(totaly.size),totaly] = 1



(100, 80000)


In [27]:
from art.attacks.evasion import ProjectedGradientDescentPyTorch

In [25]:
pgd = ProjectedGradientDescentPyTorch(modelX, norm = np.inf, eps=0.1, eps_step=0.1, max_iter=3, num_random_init=20)
x_test_pgd = pgd.generate(total.astype(np.double))




In [26]:
from art.defences.trainer import AdversarialTrainer
adv_trainer = AdversarialTrainer(modelX, attacks=pgd, ratio=1.)
adv_trainer.fit(total.astype(np.float), y=totaly.astype(np.float))

Adversarial training epochs: 100%|██████████| 20/20 [05:01<00:00, 15.06s/it]
